In [3]:
interp.load.cp(os.Path("/home/jovyan/target/scala-2.12/masterthesis-khipu_2.12-0.1.0-SNAPSHOT.jar")) 

In [5]:
import de.tmrdlt.database.workflowlist.WorkflowList
import de.tmrdlt.models.WorkflowListType

cmd3.sc:19: Symbol 'term spray.json' is missing from the classpath.
This symbol is required by 'trait de.tmrdlt.models.AdditionalJsonTypes'.
Make sure that term json is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'AdditionalJsonTypes.class' was compiled against an incompatible version of spray.
final val cmd2: _root_.ammonite.$sess.cmd2.instance.type = if (__amm_usedThings("""cmd2""")) _root_.ammonite.$sess.cmd2.instance else null
          ^Compilation Failed

: 

In [5]:
val wl = WorkflowList(
            id = 0L,
            apiId = java.util.UUID.randomUUID.toString,
            title = "bla",
            description = None,
            parentId = None,
            position = 0,
            listType = WorkflowListType.BOARD,
            state = Some(WorkflowListState.OPEN),
            dataSource = WorkflowListDataSource.Khipu,
            useCase = None,
            createdAt = LocalDateTime.now(),
            updatedAt = LocalDateTime.now()
          )

cmd5.sc:8: not found: value WorkflowListType
            listType = WorkflowListType.BOARD,
                       ^cmd5.sc:9: not found: value WorkflowListState
            state = Some(WorkflowListState.OPEN),
                         ^cmd5.sc:10: not found: value WorkflowListDataSource
            dataSource = WorkflowListDataSource.Khipu,
                         ^cmd5.sc:12: not found: value LocalDateTime
            createdAt = LocalDateTime.now(),
                        ^cmd5.sc:13: not found: value LocalDateTime
            updatedAt = LocalDateTime.now()
                        ^Compilation Failed

: 